In [1]:
import networkx as nx
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Initialise the T5 model and tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


In [3]:
# Initialize the graph
G = nx.DiGraph()

# Add nodes for Tasks, PredTasks, TasksRSRC, and RSRC
G.add_node("Task1", type="Task", duration=5, description="Excavate foundation")
G.add_node("Task2", type="Task", duration=3, description="Pour concrete")
G.add_node("Resource1", type="RSRC", description="Excavator")
G.add_node("Resource2", type="RSRC", description="Concrete Mixer")
G.add_node("Task1-Resource1", type="TasksRSRC")
G.add_node("Task2-Resource2", type="TasksRSRC")

# Add edges representing relationships
G.add_edge("Task1", "Task2", type="HAS_PREDTASK")
G.add_edge("Task1", "Resource1", type="REQUIRES_RSRC")
G.add_edge("Task2", "Resource2", type="REQUIRES_RSRC")
G.add_edge("Resource1", "Task1-Resource1", type="ALLOCATED_TO")
G.add_edge("Resource2", "Task2-Resource2", type="ALLOCATED_TO")


In [5]:
def graph_to_text(graph):
    text = "Optimize the following construction schedule:\n"
    for node, data in graph.nodes(data=True):
        text += f"Task: {node}, Duration: {data.get('duration', 'N/A')}, Description: {data.get('description', 'N/A')}\n"
    for u, v, data in graph.edges(data=True):
        text += f"{u} -> {v} ({data['type']})\n"
    return text

def optimize_schedule(graph, qa_pipeline):
    # Convert the graph to a text representation
    text_input = graph_to_text(graph)
    
    # Generate an optimized schedule using the T5 model
    output = qa_pipeline(text_input, max_length=512)
    return output[0]['generated_text']


In [7]:
optimized_schedule = optimize_schedule(G, qa_pipeline)
print(optimized_schedule)


: Excavator Task: Resource1, Duration: 5, Description: Excavate foundation Task: Task2, Duration: 3, Description: Pour concrete Task: Resource1, Duration: N/A, Description: Pour concrete Task: Resource1, Duration: N/A, Description: Pour concrete Task: Resource2, Duration: N/A, Description: N/A Task: Task2-Resource2, Duration: N/A, Description: N/A Task: Task1-Resource2, Duration: N/A, Description: N/A Task:
